### **MAESTRÍA EN INTELIGENCIA ARTIFICIAL APLICADA**

#### **Curso: TC4029 - Ciencia y analítica de datos**

Tecnológico de Monterrey

Prof Grettel Barceló Alonso

**Semana 7**
Análisis de Componentes Principales (PCA)

---

*   **NÚMERO DE EQUIPO:**
*     Equipo #83

---        
        
*   **MIEMBROS DEL EQUIPO:**
*     Ernesto Damián Favela Vela: A00738581
*     Gustavo Alejandro Garduño Macedo: A01795341
*     Julia Gabriela Pinedo: A01795315
*     Juan Manuel Rodríguez Mateos: A01794890

---

In [2]:
# Importar Librerías

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from tabulate import tabulate

In [4]:
# Cargar el DataFrame

cereals_df = pd.read_csv('../data/Cereals.csv')
cereals_df

,name,mfr,type,calories,protein,fat,sodium,fiber,carbo,sugars,potass,vitamins,shelf,weight,cups,rating
0,100%_Bran,N,C,70,4,1,130,10.0,5.0,6.0,280.0,25,3,1.0,0.33,68.402973
1,100%_Natural_Bran,Q,C,120,3,5,15,2.0,8.0,8.0,135.0,0,3,1.0,1.00,33.983679
2,All-Bran,K,C,70,4,1,260,9.0,7.0,5.0,320.0,25,3,1.0,0.33,59.425505
3,All-Bran_with_Extra_Fiber,K,C,50,4,0,140,14.0,8.0,0.0,330.0,25,3,1.0,0.50,93.704912
4,Almond_Delight,R,C,110,2,2,200,1.0,14.0,8.0,NaN,25,3,1.0,0.75,34.384843
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,Triples,G,C,110,2,1,250,0.0,21.0,3.0,60.0,25,3,1.0,0.75,39.106174
73,Trix,G,C,110,1,1,140,0.0,13.0,12.0,25.0,25,2,1.0,1.00,27.753301
74,Wheat_Chex,R,C,100,3,1,230,3.0,17.0,3.0,115.0,25,1,1.0,0.67,49.787445
75,Wheaties,G,C,100,3,1,200,3.0,17.0,3.0,110.0,25,1,1.0,1.00,51.592193


# **Parte 1**. EDA y preprocesamiento

1. Obtén estadísticas descriptivas para todas las variables del dataframe.

In [ ]:
# *************************** PARTE 0: Clasificación de estadísticas *****************************

# Opcional: Se creará una función para etiquetar las variables de acuerdo con la categoría a la
# que pertenecen, para visualizarlos en una tabla.

def separate_variables_by_type(df):
    """
    Separar las columnas entre numéricas (dtype= int, float) y cualitativas (dtype= object)

    Args:
        df (DataFrame)

    Returns:
        numeric_df (tabla impresa clasificando a las variables numéricas)
        object_df (tabla impresa clasificando a las variables categóricas)
    """
    numeric_columns = df.select_dtypes(include=['int', 'float']).columns
    object_columns = df.select_dtypes(include=['object']).columns

    # Se declara una lista vacía, la cual se utilizará para almacenar los datos en el dataframe
    # que se generará
    data_types = []

    # Ciclo "for" obtener los data types de cada una de las columnas
    for col in df.columns:
        if col in numeric_columns:
            data_type = "Numeric" # Etiqueta para el tipo "numérico"
        elif col in object_columns:
            data_type = "Object" # Etiqueta para el tipo "objeto"
        data_types.append((col, data_type)) # Guarda las etiquetas de las columnas en la lista

    # Crea un DataFrame para las columnas numéricas
    numeric_df = pd.DataFrame([i for i in data_types if i[1] == "Numeric"],
                                   columns=["Variable", "Categoría"])
    # Crea un DataFrame para las columnas de texto
    object_df = pd.DataFrame([i for i in data_types if i[1] == "Object"],
                                  columns=["Variable", "Categoría"])

    # Imprimir las tablas con los datos
    print('Columnas del tipo "Numeric":')
    print(tabulate(numeric_df, headers="keys", tablefmt="psql", showindex=True))
    print('\n\n')
    print('Columnas del tipo "Object":')
    print(tabulate(object_df, headers="keys", tablefmt="psql", showindex=True))

In [ ]:
# Se imprimen las columnas separadas por categoría.

columns_separated = separate_variables_by_type(df=cereals_df)

Columnas del tipo "Numeric":
+----+------------+-------------+
|    | Variable   | Categoría   |
|----+------------+-------------|
|  0 | calories   | Numeric     |
|  1 | protein    | Numeric     |
|  2 | fat        | Numeric     |
|  3 | sodium     | Numeric     |
|  4 | fiber      | Numeric     |
|  5 | carbo      | Numeric     |
|  6 | sugars     | Numeric     |
|  7 | potass     | Numeric     |
|  8 | vitamins   | Numeric     |
|  9 | shelf      | Numeric     |
| 10 | weight     | Numeric     |
| 11 | cups       | Numeric     |
| 12 | rating     | Numeric     |
+----+------------+-------------+



Columnas del tipo "Object":
+----+------------+-------------+
|    | Variable   | Categoría   |
|----+------------+-------------|
|  0 | name       | Object      |
|  1 | mfr        | Object      |
|  2 | type       | Object      |
+----+------------+-------------+


In [ ]:
# ****************************** PARTE 1: Resumen de estadísticas ********************************

"""Estadísticas descriptivas para las variables numéricas del DataFrame"""

# Se declara la función ".describe()" para obtener los datos estadísticos de las columnas que
# se clasificaron como "numéricas".

numeric_columns = cereals_df.select_dtypes(include=['int', 'float']).columns

numerical_summary = cereals_df[numeric_columns].describe()
numerical_summary.T

,count,mean,std,min,25%,50%,75%,max
calories,77.0,106.883117,19.484119,50.000000,100.000000,110.000000,110.000000,160.000000
protein,77.0,2.545455,1.094790,1.000000,2.000000,3.000000,3.000000,6.000000
fat,77.0,1.012987,1.006473,0.000000,0.000000,1.000000,2.000000,5.000000
sodium,77.0,159.675325,83.832295,0.000000,130.000000,180.000000,210.000000,320.000000
fiber,77.0,2.151948,2.383364,0.000000,1.000000,2.000000,3.000000,14.000000
carbo,76.0,14.802632,3.907326,5.000000,12.000000,14.500000,17.000000,23.000000
sugars,76.0,7.026316,4.378656,0.000000,3.000000,7.000000,11.000000,15.000000
potass,75.0,98.666667,70.410636,15.000000,42.500000,90.000000,120.000000,330.000000
vitamins,77.0,28.246753,22.342523,0.000000,25.000000,25.000000,25.000000,100.000000
shelf,77.0,2.207792,0.832524,1.000000,1.000000,2.000000,3.000000,3.000000


In [ ]:
"""Estadísticas descriptivas para las variables numéricas del DataFrame"""

# Se obtienen los datos estadísticos de las columnas clasificadas como "categóricas".

object_columns = cereals_df.select_dtypes(include=['object']).columns

object_summary = cereals_df[object_columns].describe()
object_summary.T

,count,unique,top,freq
name,77,77,100%_Bran,1
mfr,77,7,K,23
type,77,2,C,74


In [ ]:
# NOTA: A pesar de que la variable "shelf" se clasificó como una variable numérica, esta en
# realidad una variable cualitativa ordinal, debido a que clasifica el estante donde se
# encuentra exhibido el cereal (contando desde el piso). En puntos posteriores, se le dará
# el tratamiento correspondiente al tipo de variable.

In [ ]:
"""Ejemplo de estadísticas descriptivas obtenidas para la variable: shelf"""

# Copia del DF original para no afectarlo.
cereals_df_copy = cereals_df.copy()

# Nombrar las categorías y reemplazarlas en el DF copia
shelf_categories = {1: 'Shelf level 1', 2: 'Shelf level 2', 3: 'Shelf level 3'}
cereals_df_copy['shelf'] = cereals_df_copy['shelf'].map(shelf_categories)

# Obtener estadísticas descriptivas
shelf_description = cereals_df_copy['shelf'].describe()
shelf_description.T

count                77
unique                3
top       Shelf level 3
freq                 36
Name: shelf, dtype: object

Genera algunos gráficos para familiarizarte con el conjunto de datos. Al menos deberás incluir los que te permitan responder los siguientes cuestionamientos:

**Nota.** Debes responder de manera explícita las preguntas, apoyándote de los resultados observados de cada gráfico.

2a) ¿Cuál es la frecuencia o conteo de cada categoría para las variables: `mfr` y `shelf` (por separado)?

2b) Combina los resultados previos para observar, de cada fabricante, cuántos productos tiene en cada estante.

 3a) ¿Cuál es el rango más frecuente de la variable `rating`?

3b) ¿Cuáles son los 10 cereales mejor evaluados y a qué fabricante corresponden?

4a) Compara la distribución de las calorías según el fabricante, por medio de boxplots.

4b) ¿Cuáles son las dos variables que tienen mayor correlación con `rating`?

5. Elimina todos los registros con algún dato faltante y reinicia el índice del dataframe para que quede con valores consecutivos.

In [ ]:
# ************************** PARTE 0: Porcentaje de Valores Faltantes ****************************

# Opcional: Se creará una función para obtener automáticamente el porcentaje de valores faltantes
# por cada variable del DataFrame.

def find_missing_values_per_column(df):
    """
    Obtener el porcentaje de valores faltantes por columna e imprimirlo en una tabla

    Args:
        df (DataFrame)

    Returns:
        df_missing (tabla impresa con los porcentajes de valores faltantes por columna)
    """
    missing_percentage = df.isna().sum() * 100 / len(df)

    # Guardar en un DataFrame para su impresión en una tabla

    df_missing = pd.DataFrame({
        'Columna': df.columns,
        'Porcentaje Faltantes': missing_percentage.astype(str) + '%'
        # Imprime el valor con el símbolo "%"
    })

    # Imprimir una tabla con los datos
    print(tabulate(df_missing, headers='keys', tablefmt='psql', showindex=False))

In [ ]:
# Se imprime la tabla de valores faltantes.

missing_values = find_missing_values_per_column(df=cereals_df)

+-----------+------------------------+
| Columna   | Porcentaje Faltantes   |
|-----------+------------------------|
| name      | 0.0%                   |
| mfr       | 0.0%                   |
| type      | 0.0%                   |
| calories  | 0.0%                   |
| protein   | 0.0%                   |
| fat       | 0.0%                   |
| sodium    | 0.0%                   |
| fiber     | 0.0%                   |
| carbo     | 1.2987012987012987%    |
| sugars    | 1.2987012987012987%    |
| potass    | 2.5974025974025974%    |
| vitamins  | 0.0%                   |
| shelf     | 0.0%                   |
| weight    | 0.0%                   |
| cups      | 0.0%                   |
| rating    | 0.0%                   |
+-----------+------------------------+


In [ ]:
# Se observa que las columnas que tienen un porcentaje de valores faltantes son: carbo, sugars y
# potass. Se guardarán en la lista "columns_with_missing_values" para eliminar los registros que
# contengan valores faltantes en dichas columnas.

# ************************** PARTE 1: Eliminación de Valores Faltantes ***************************

columns_with_missing_values = ['carbo', 'sugars', 'potass']

# Eliminar valores faltantes.
cereals_df = cereals_df.dropna(subset=columns_with_missing_values)
cereals_df

,name,mfr,type,calories,protein,fat,sodium,fiber,carbo,sugars,potass,vitamins,shelf,weight,cups,rating
0,100%_Bran,N,C,70,4,1,130,10.0,5.0,6.0,280.0,25,3,1.0,0.33,68.402973
1,100%_Natural_Bran,Q,C,120,3,5,15,2.0,8.0,8.0,135.0,0,3,1.0,1.00,33.983679
2,All-Bran,K,C,70,4,1,260,9.0,7.0,5.0,320.0,25,3,1.0,0.33,59.425505
3,All-Bran_with_Extra_Fiber,K,C,50,4,0,140,14.0,8.0,0.0,330.0,25,3,1.0,0.50,93.704912
5,Apple_Cinnamon_Cheerios,G,C,110,2,2,180,1.5,10.5,10.0,70.0,25,1,1.0,0.75,29.509541
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,Triples,G,C,110,2,1,250,0.0,21.0,3.0,60.0,25,3,1.0,0.75,39.106174
73,Trix,G,C,110,1,1,140,0.0,13.0,12.0,25.0,25,2,1.0,1.00,27.753301
74,Wheat_Chex,R,C,100,3,1,230,3.0,17.0,3.0,115.0,25,1,1.0,0.67,49.787445
75,Wheaties,G,C,100,3,1,200,3.0,17.0,3.0,110.0,25,1,1.0,1.00,51.592193


In [ ]:
# Se reinicia el índice del DataFrame para mantener la continuidad entre las columnas

cereals_df = cereals_df.reset_index(drop=True)
cereals_df

,name,mfr,type,calories,protein,fat,sodium,fiber,carbo,sugars,potass,vitamins,shelf,weight,cups,rating
0,100%_Bran,N,C,70,4,1,130,10.0,5.0,6.0,280.0,25,3,1.0,0.33,68.402973
1,100%_Natural_Bran,Q,C,120,3,5,15,2.0,8.0,8.0,135.0,0,3,1.0,1.00,33.983679
2,All-Bran,K,C,70,4,1,260,9.0,7.0,5.0,320.0,25,3,1.0,0.33,59.425505
3,All-Bran_with_Extra_Fiber,K,C,50,4,0,140,14.0,8.0,0.0,330.0,25,3,1.0,0.50,93.704912
4,Apple_Cinnamon_Cheerios,G,C,110,2,2,180,1.5,10.5,10.0,70.0,25,1,1.0,0.75,29.509541
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,Triples,G,C,110,2,1,250,0.0,21.0,3.0,60.0,25,3,1.0,0.75,39.106174
70,Trix,G,C,110,1,1,140,0.0,13.0,12.0,25.0,25,2,1.0,1.00,27.753301
71,Wheat_Chex,R,C,100,3,1,230,3.0,17.0,3.0,115.0,25,1,1.0,0.67,49.787445
72,Wheaties,G,C,100,3,1,200,3.0,17.0,3.0,110.0,25,1,1.0,1.00,51.592193


In [ ]:
# Se vuelve a imprimir la tabla de valores faltantes para comprobar que la operación
# se realizó correctamente

missing_values_after_drop = find_missing_values_per_column(df=cereals_df)

+-----------+------------------------+
| Columna   | Porcentaje Faltantes   |
|-----------+------------------------|
| name      | 0.0%                   |
| mfr       | 0.0%                   |
| type      | 0.0%                   |
| calories  | 0.0%                   |
| protein   | 0.0%                   |
| fat       | 0.0%                   |
| sodium    | 0.0%                   |
| fiber     | 0.0%                   |
| carbo     | 0.0%                   |
| sugars    | 0.0%                   |
| potass    | 0.0%                   |
| vitamins  | 0.0%                   |
| shelf     | 0.0%                   |
| weight    | 0.0%                   |
| cups      | 0.0%                   |
| rating    | 0.0%                   |
+-----------+------------------------+


# **Parte 2**. Ingeniería de características

6. Considerando que `rating` es la variable de salida, almacénala en una variable `y` y separa los predictores **numéricos** en `X`. Escala los valores de `X` y aplica `PCA` para proyectar los datos en el nuevo espacio de vectores.

7. Obtén la curva del porcentaje de varianza acumulada y determine el número mínimo de componentes principales que explique más del 90% de la varianza. Imprime la información de dichos componentes.

8. ¿Cuáles son las tres variables más importantes en el cálculo del primer componente?


9. ¿Todas las variables categóricas son predictores? Aplica una técnica de encoding a las que sí lo sean. Justifica la elección de tu técnica.

10. Conjunta, en un dataframe, las valores proyectados en los componentes seleccionados (mínimo), las transformaciones obtenidas de las variables categóricas que serán predictores y la variable de salida. Almacena el dataframe resultante en archivo.